 Install and Import Required Libraries

In [1]:
import numpy as np
import random
import json
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.stem import WordNetLemmatizer

# Download required NLTK data
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/wordnet')
    nltk.data.find('corpora/omw-1.4')
except LookupError:
    print("Downloading NLTK data...")
    nltk.download('punkt')
    nltk.download('wordnet')
    nltk.download('omw-1.4')
    print("Download complete!")

lemmatizer = WordNetLemmatizer()

Download complete!


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\geeky\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\geeky\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\geeky\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Knowledge Base (Training Data)

In [2]:
knowledge_base = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Hi", "Hello", "Hey", "How are you?", "What's up?"],
            "responses": ["Hello! How can I help you today?", "Hi there! What can I do for you?", "Greetings! How may I assist you?"]
        },
        {
            "tag": "goodbye",
            "patterns": ["Bye", "See you later", "Goodbye", "Take care"],
            "responses": ["Goodbye! Have a great day!", "See you later!", "Until next time!"]
        },
        {
            "tag": "thanks",
            "patterns": ["Thanks", "Thank you", "That's helpful", "Appreciate it"],
            "responses": ["Happy to help!", "Anytime!", "My pleasure!"]
        },
        {
            "tag": "identity",
            "patterns": ["What is your name?", "Who are you?", "Tell me about yourself"],
            "responses": [
                "I'm ChatBot, your AI assistant for this project.",
                "You can call me CB! I'm here to help with your questions.",
                "I'm an AI chatbot created for this internship project."
            ]
        },
        {
            "tag": "capabilities",
            "patterns": ["What can you do?", "How can you help me?", "What are your functions?"],
            "responses": [
                "I can answer questions, help with documentation, and assist with school projects.",
                "My capabilities include information lookup, basic conversation, and project guidance.",
                "I can help you prepare documents, brainstorm ideas, and answer your questions."
            ]
        },
        {
            "tag": "project_help",
            "patterns": [
                "Can you help me with my school project?",
                "Help me prepare a document",
                "I need project assistance",
                "Can you draft something for my assignment?"
            ],
            "responses": [
                "Sure! What topic is your project about? I can help with research and structure.",
                "I'd be happy to help with your school project. What specific help do you need?",
                "For project documents, I can suggest outlines and key points. What's your subject?"
            ]
        },
        {
            "tag": "document_types",
            "patterns": [
                "What documents can you help with?",
                "Types of documents you know",
                "Can you make reports/presentations?"
            ],
            "responses": [
                "I can assist with reports, essays, presentations, and basic research papers.",
                "I can help structure documents like project reports, literature reviews, and simple presentations.",
                "My document support includes outlines, structure suggestions, and content ideas for school projects."
            ]
        },
        {
            "tag": "fallback",
            "patterns": [],
            "responses": [
                "I'm not sure I understand. Could you rephrase that?",
                "I don't have an answer for that yet. Try asking something else.",
                "I'm still learning. Could you ask me about project help or my capabilities?"
            ]
        }
    ]
}

# Save knowledge base
with open('knowledge_base.json', 'w') as file:
    json.dump(knowledge_base, file)

Data Preprocessing (Text Processing)

In [4]:
def preprocess_text(text):
    try:
        # Convert to lowercase and remove punctuation
        text = text.lower().translate(str.maketrans('', '', string.punctuation))
        # Tokenize and lemmatize
        tokens = nltk.word_tokenize(text)
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        return ' '.join(tokens)
    except Exception as e:
        print(f"Error in preprocessing: {e}")
        return ""


Model Training and Preparing Data

In [6]:
tags = []
patterns = []
responses = {}

for intent in knowledge_base['intents']:
    tag = intent['tag']
    tags.append(tag)
    responses[tag] = intent['responses']
    for pattern in intent['patterns']:
        patterns.append((preprocess_text(pattern), tag))

X_train = [pattern[0] for pattern in patterns]
y_train = [pattern[1] for pattern in patterns]

# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)

In [8]:
def get_response(user_input):
    processed_input = preprocess_text(user_input)
    
    if not processed_input:
        return random.choice(responses['fallback'])
    
    input_vector = vectorizer.transform([processed_input])
    similarities = cosine_similarity(input_vector, X_train_vectorized)
    
    if len(similarities[0]) == 0:
        return random.choice(responses['fallback'])
    
    most_similar_idx = np.argmax(similarities)
    similarity_score = similarities[0][most_similar_idx]
    
    # Threshold for responding
    if similarity_score > 0.5:
        predicted_tag = y_train[most_similar_idx]
        return random.choice(responses[predicted_tag])
    else:
        return random.choice(responses['fallback'])

Chat Interface

In [10]:
def chat():
    print("\nChatBot: Hello! I'm your AI assistant. Type 'quit' to exit.")
    print("You can ask me about:\n- My capabilities\n- School project help\n- Document preparation\n")
    
    while True:
        user_input = input("You: ")
        
        if user_input.lower() == 'quit':
            print("ChatBot: Goodbye! Come back if you need more help.")
            break
            
        response = get_response(user_input)
        print("ChatBot:", response)

Test Prompts

In [11]:
test_prompts = [
    "Hello!",
    "What is your name?",
    "What can you do for me?",
    "Can you help me with my school project?",
    "I need to prepare a science project report",
    "What types of documents can you help with?",
    "Can you make a PowerPoint presentation?",
    "Thank you for your help",
    "Goodbye"
]

print("\n=== TESTING CHATBOT ===")
for prompt in test_prompts:
    print(f"\nYou: {prompt}")
    print(f"ChatBot: {get_response(prompt)}")



=== TESTING CHATBOT ===

You: Hello!
ChatBot: Greetings! How may I assist you?

You: What is your name?
ChatBot: I'm an AI chatbot created for this internship project.

You: What can you do for me?
ChatBot: My capabilities include information lookup, basic conversation, and project guidance.

You: Can you help me with my school project?
ChatBot: Sure! What topic is your project about? I can help with research and structure.

You: I need to prepare a science project report
ChatBot: For project documents, I can suggest outlines and key points. What's your subject?

You: What types of documents can you help with?
ChatBot: I can help structure documents like project reports, literature reviews, and simple presentations.

You: Can you make a PowerPoint presentation?
ChatBot: My document support includes outlines, structure suggestions, and content ideas for school projects.

You: Thank you for your help
ChatBot: My pleasure!

You: Goodbye
ChatBot: See you later!
